In [ ]:
#testing only

import os
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from dotenv import load_dotenv
load_dotenv()

ENV = os.getenv("ENV")
MODEL = os.getenv("MODEL")
print(ENV, MODEL)

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)
model.invoke("Tell me a joke in 10 words")

dev llama3.2:1b


/var/folders/25/5bzk4lrj6791krj2dqh29y300000gn/T/ipykernel_864/2088745703.py:11: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model=MODEL)
/var/folders/25/5bzk4lrj6791krj2dqh29y300000gn/T/ipykernel_864/2088745703.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model=MODEL)


'A man walked into a library and asked the librarian, "Do you have any books on Pavlov?"'

In [ ]:
# testing only
from youtube_transcript_api import YouTubeTranscriptApi            
transcript = YouTubeTranscriptApi.get_transcript("fE2sunDZhzg")
full_text = " ".join([entry['text'] for entry in transcript])
print(full_text)

there are a million very different flatbreads they call Focaccia in Italy this is not one of them this is my own little Focaccia my goal was to make one that's flavorful enough to eat all by itself this dough holds a secret mini sheet pans are a great thing for the home kitchen they fit in the dishwasher and I don't need a whole sheet Pan's worth of bread I just need a little for that much I'll start with a cup of water 237 mils into that goes a teaspoon of dry yeast which is more than you'd normally use for Focaccia but I just really like the taste of extra fermentation summon forth the upside down bear or put in a little spoonful of any form of sugar mostly for flavor the yeast don't really need it now I've got some pecorino cheese but any hard grating cheese will do I'm gonna finely grate a little pile maybe 30 grams or thereabouts and yep I'm putting it directly inside my bread dough don't do much more than that or else it lessens your rise either chemically or physically I'm not s

In [ ]:
#create post

import os
from typing import List, Dict
from youtube_transcript_api import YouTubeTranscriptApi
import faiss
import numpy as np
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import textwrap

from dotenv import load_dotenv
load_dotenv()
ENV = os.getenv("ENV")
MODEL = os.getenv("MODEL")
print(ENV, MODEL)

class ContentGenerator:
    def __init__(self):
        try:
            # Initialize Ollama
            self.llm = Ollama(model=MODEL)
            self.embeddings = OllamaEmbeddings(model=MODEL)
            
            # Get actual embedding dimension by embedding a test string
            print("Testing embeddings...")
            test_embedding = self.embeddings.embed_query("test")
            self.dimension = len(test_embedding)
            print(f"Embedding dimension: {self.dimension}")
            
            # Initialize FAISS index with correct dimension
            self.index = faiss.IndexFlatL2(self.dimension)
            self.stored_texts = []
        except Exception as e:
            print(f"Error during initialization: {e}")
            raise


    def get_youtube_transcript(self, video_id: str) -> str:
        """Fetch and process YouTube transcript"""
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            full_text = " ".join([entry['text'] for entry in transcript])
            return full_text
        except Exception as e:
            print(f"Error fetching transcript: {e}")
            return ""

    def process_transcript(self, text: str) -> List[str]:
        """Split transcript into chunks"""
        return textwrap.wrap(text, 2000)  # Split into 2000-character chunks

    def index_content(self, chunks: List[str]):
        """Index content chunks using FAISS"""
        for i, chunk in enumerate(chunks):
            try:
                print(f"Processing chunk {i+1}/{len(chunks)}")
                # Get embedding for chunk
                embedding = self.embeddings.embed_query(chunk)
                # Reshape embedding to 2D array
                embedding_array = np.array([embedding], dtype='float32')
                # Add to FAISS index
                self.index.add(embedding_array)
                self.stored_texts.append(chunk)
            except Exception as e:
                print(f"Error indexing chunk {i+1}: {e}")
                continue

    def generate_social_posts(self, video_id: str, num_posts: int = 3) -> Dict[str, List[str]]:
        """Generate social media posts from video content"""
        # Get transcript
        transcript = self.get_youtube_transcript(video_id)
        if not transcript:
            return {"error": ["Could not fetch transcript"]}

        # Process and index content
        chunks = self.process_transcript(transcript)
        print("chunks created")
        self.index_content(chunks)

        # Generate posts
        twitter_posts = []
        linkedin_posts = []

        for _ in range(num_posts):
            # Twitter prompt
            twitter_prompt = f"""
            Based on this content, create an engaging tweet (max 280 characters):
            {chunks[0][:500]}...
            
            Make it conversational and include relevant hashtags. Don't use generic phrases.
            """
            
            # LinkedIn prompt
            linkedin_prompt = f"""
            Based on this content, create an engaging LinkedIn post (200-400 words):
            {chunks[0][:1000]}...
            
            Include:
            - An attention-grabbing opening
            - Key insights from the video
            - A call to action
            Make it professional but conversational. Add relevant hashtags at the end.
            """

            twitter_post = self.llm.invoke(twitter_prompt)
            linkedin_post = self.llm.invoke(linkedin_prompt)

            twitter_posts.append(twitter_post)
            linkedin_posts.append(linkedin_post)

        return {
            "twitter": twitter_posts,
            "linkedin": linkedin_posts
        }

def main():
    # Example usage
    generator = ContentGenerator()
    video_id = "fE2sunDZhzg"  # Example: "dQw4w9WgXcQ"
    
    posts = generator.generate_social_posts(video_id)
    
    print("\nTwitter Posts:")
    for i, post in enumerate(posts["twitter"], 1):
        print(f"\nPost {i}:")
        print(post)
        print("-" * 50)
    
    print("\nLinkedIn Posts:")
    for i, post in enumerate(posts["linkedin"], 1):
        print(f"\nPost {i}:")
        print(post)
        print("-" * 50)

if __name__ == "__main__":
    main()

dev llama3.2:1b
Testing embeddings...
Embedding dimension: 2048
chunks created
Processing chunk 1/4
Processing chunk 2/4
Processing chunk 3/4
Processing chunk 4/4

Twitter Posts:

Post 1:
"just tried to make my own focaccia this morning, but let's just say it's a work in progress - secret ingredient? mini sheet pans to the rescue? how do these Italian bakers keep their dough from falling flat? #focaccianowhate #breadwinner #homemadewithlove #Italianfoodlover #bakingstruggles"
--------------------------------------------------

Post 2:
"just tried my hand at making Focaccia, but let's be real - there's only one version: the one I whipped up myself in under 20 min! secret ingredient? patience #flatbreadlove #fockacalove #homemakinghacks #italianinspo #bakefromscratch"
--------------------------------------------------

Post 3:
"Who needs the whole shebang when you can whip up a mini Focaccia? Just mix & bake 1 cup water, 237ml yeast, flour... I'm talking tiny sheets that fit in my dishwa

In [29]:
# Generate IPO analysis
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Tuple
import os
from youtube_transcript_api import YouTubeTranscriptApi
import faiss
import numpy as np
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
import textwrap
import time
import re
from datetime import datetime, timedelta
from googleapiclient.discovery import build

class IPOAnalyzer:
        # If modifying scopes, delete the token.pickle file
    SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']
    API_SERVICE_NAME = 'youtube'
    API_VERSION = 'v3'

    def __init__(self, youtube_api_key, model_name="llama3.2:1b"):
        # Initialize Ollama
        print(f"Initializing with model: {model_name}")
        self.youtube_api_key=youtube_api_key
        self.llm = Ollama(model=model_name)
        self.embeddings = OllamaEmbeddings(model=model_name)
        
        # Initialize FAISS
        test_embedding = self.embeddings.embed_query("test")
        self.dimension = len(test_embedding)
        self.index = faiss.IndexFlatL2(self.dimension)
        self.stored_texts = []
    def build_service(self):
        """
        Build YouTube API service with either OAuth credentials or API key
        """
        if self.youtube_api_key:
            return build(self.API_SERVICE_NAME, self.API_VERSION, developerKey=self.youtube_api_key)
        else:
            raise ValueError("Neither valid credentials nor API key available")
        
    def extract_video_id(self, url: str) -> str:
        """Extract video ID from YouTube URL"""
        patterns = [
            r'(?:v=|\/)([0-9A-Za-z_-]{11}).*',
            r'(?:embed\/)([0-9A-Za-z_-]{11})',
            r'(?:watch\?v=)([0-9A-Za-z_-]{11})'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, url)
            if match:
                return match.group(1)
        return None

    def search_videos(self, query: str, max_results: int = 10) -> List[Dict]:
        """
        Search for YouTube videos
        
        Args:
            query: Search query
            max_results: Maximum number of results to return
            
        Returns:
            List of dictionaries containing video information
        """
        service = self.build_service()
        
        request = service.search().list(
            part="snippet",
            maxResults=max_results,
            q=query,
            type="video"
        )
        
        response = request.execute()
        
        videos = []
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            snippet = item['snippet']
            
            videos.append({
                'video_id': video_id,
                'title': snippet['title'],
                'description': snippet['description'],
                'published_at': snippet['publishedAt'],
                'channel_title': snippet['channelTitle'],
                'thumbnail_url': snippet['thumbnails']['default']['url'],
                'url': f'https://www.youtube.com/watch?v={video_id}'
            })
            
        return videos

    def get_youtube_transcript(self, video_id: str) -> str:
        """Fetch and process YouTube transcript"""
        try:
            # transcript = YouTubeTranscriptApi.get_transcript(video_id)
            # full_text = " ".join([entry['text'] for entry in transcript])
            # print(f"Successfully fetched transcript of length: {len(full_text)}")
            transcript_list = YouTubeTranscriptApi.list_transcripts("1h0fxQubJKk")
            for transcript in transcript_list:
                if transcript.language_code == 'en':
                    trn = transcript.fetch()
                    full_text = " ".join([entry['text'] for entry in trn])
                    return full_text
                if transcript.is_translatable and transcript.language_code != 'en':
                    trn = transcript.translate('en').fetch()
                    full_text = " ".join([entry['text'] for entry in trn])
                    return full_text
        except Exception as e:
            print(f"Error fetching transcript for video {video_id}: {e}")
            return ""

    def analyze_ipo(self, ipo_name: str) -> Dict:
        """Analyze IPO based on multiple video reviews"""
        try:
            # Search for relevant videos
            print(f"Searching videos for {ipo_name} IPO...")
            videos = self.search_videos(ipo_name)
            if not videos:
                return {"error": "No videos found"}

            all_transcripts = []
            
            # Collect transcripts from all videos
            for video in videos:
                try:
                    transcript = self.get_youtube_transcript(video['video_id'])
                    if transcript:
                        all_transcripts.append({
                            'text': transcript,
                            'video': video
                        })
                        print(f"Successfully processed video: {video['title']}")
                except Exception as e:
                    print(f"Error getting transcript for video {video['video_id']}: {e}")
                    continue

            if not all_transcripts:
                return {"error": "Could not get any transcripts"}

            # Analyze consolidated information
            analysis_prompt = f"""
            Analyze these {len(all_transcripts)} video transcripts about {ipo_name} IPO and provide:
            1. Overall recommendation (Subscribe/Avoid/Neutral)
            2. Key strengths (max 3)
            3. Key risks (max 3)
            4. GMP (Grey Market Premium) trends if mentioned
            5. Valuation assessment
            6. Subscription status if mentioned
            7. Price band
            8. Issue size
            9. Lot size
            10. Important dates

            Here are the transcripts:
            {' '.join([t['text'][:2000] + '...' for t in all_transcripts])}
            
            Format the response in clear sections with headings.
            """

            print("Generating IPO analysis...")
            analysis = self.llm.invoke(analysis_prompt)

            # Generate social media content
            social_content = self.generate_social_content(ipo_name, analysis)

            return {
                "ipo_name": ipo_name,
                "analysis": analysis,
                "social_content": social_content,
                "sources": [{"title": t['video']['title'], "url": t['video']['url']} for t in all_transcripts]
            }

        except Exception as e:
            print(f"Error in analyze_ipo: {e}")
            return {"error": str(e)}

    def generate_social_content(self, ipo_name: str, analysis: str) -> Dict:
        """Generate social media content based on IPO analysis"""
        try:
            # Twitter post
            twitter_prompt = f"""
            Create an engaging tweet (max 280 chars) about {ipo_name} IPO based on this analysis:
            {analysis[:500]}
            
            Include:
            - Clear recommendation
            - 1 key strength or risk
            - GMP if available
            - Hashtags: #IPO #{ipo_name}IPO
            Be direct and professional.
            """

            # WhatsApp message
            whatsapp_prompt = f"""
            Create a clear, concise WhatsApp message about {ipo_name} IPO based on this analysis:
            {analysis[:500]}
            
            Include:
            - Recommendation
            - Price band
            - Important dates
            - 2 key points
            - GMP if available
            - Brief conclusion
            
            Format with emojis and bullet points for easy reading.
            Keep it informal but informative.
            Max 200 words.
            """

            return {
                "twitter": self.llm.invoke(twitter_prompt),
                "whatsapp": self.llm.invoke(whatsapp_prompt)
            }

        except Exception as e:
            print(f"Error generating social content: {e}")
            return {"error": str(e)}

def main():
    try:
        # Initialize analyzer
        analyzer = IPOAnalyzer(os.getenv("YOUTUBE_API_KEY"), model_name="llama3.2:1b")
        
        # Example usage
        ipo_name = "Quadrant"  # Replace with your IPO name
        print(f"Analyzing {ipo_name} IPO...")
        
        result = analyzer.analyze_ipo(ipo_name)
        
        if "error" in result:
            print("Error:", result["error"])
            return

        # Print results
        print("\nIPO Analysis:")
        print("-" * 50)
        print(result["analysis"])
        
        print("\nTwitter Post:")
        print("-" * 50)
        print(result["social_content"]["twitter"])
        
        print("\nWhatsApp Message:")
        print("-" * 50)
        print(result["social_content"]["whatsapp"])
        
        print("\nSources:")
        print("-" * 50)
        for source in result["sources"]:
            print(f"- {source['title']}")
            print(f"  {source['url']}")
            
    except Exception as e:
        print(f"Error in main: {e}")

if __name__ == "__main__":
    main()

Initializing with model: llama3.2:1b
Analyzing Quadrant IPO...
Searching videos for Quadrant IPO...
Successfully processed video: Quadrant IPO Listing Day Strategy | Laxmi IPO Anchor List | Standard IPO | Jayesh Khatri
Successfully processed video: Tesla Cybertruck Autopilot For 100 Hours
Successfully processed video: Standard Glass Lining IPO Listing Day Strategy | Quadrant Future Tek IPO Hold or sell ? | Ankur
Successfully processed video: quadrant future tek ipo refund💥standard glass ipo listing day strategy🤑Laxmi dental ipo gmp today💥
Successfully processed video: We Tested Real F1 Tires Off Road!
Successfully processed video: Quadrant Future Tek IPO Full Review | Quadrant Future Tek IPO GMP | Apply Or Not?
Successfully processed video: Lando Norris Guesses The McLaren Driver
Successfully processed video: WE MADE OUR OWN QUADRANT FORMULA 1 TRACK
Successfully processed video: Quadrant Future Tek Ipo Allotment Chances 🔴Quadrant Future Tek Ipo🔴Quadrant Future Tek Ipo Gmp Today
Success